# Gold Meta-Model Training - Attempt 1

**Self-contained training notebook**

This notebook:
1. Fetches base features from yfinance and FRED
2. Generates submodel outputs using saved HMM parameters
3. Merges all 39 features
4. Trains XGBoost meta-model with custom directional-weighted MAE objective
5. Optimizes hyperparameters via Optuna (50 trials)
6. Evaluates on test set against all 4 targets

**Architecture**: XGBoost with directional-weighted MAE loss

**Features**: 19 base + 20 submodel outputs = 39 total

**Targets**: DA > 56%, HC-DA > 60%, MAE < 0.75%, Sharpe > 0.8

In [ ]:
# === IMPORTS ===
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Install packages if needed
try:
    import xgboost as xgb
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "xgboost"], check=True)
    import xgboost as xgb

try:
    import optuna
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "optuna"], check=True)
    import optuna

try:
    import yfinance as yf
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "yfinance"], check=True)
    import yfinance as yf

try:
    from fredapi import Fred
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "fredapi"], check=True)
    from fredapi import Fred

try:
    from hmmlearn import hmm as hmmlearn_hmm
except ImportError:
    import subprocess
    subprocess.run(["pip", "install", "hmmlearn"], check=True)
    from hmmlearn import hmm as hmmlearn_hmm

from sklearn.feature_selection import mutual_info_regression

print("All imports successful!")
print(f"XGBoost version: {xgb.__version__}")
print(f"Optuna version: {optuna.__version__}")

# Random seeds
np.random.seed(42)

# Get FRED API key from Kaggle Secrets
try:
    from kaggle_secrets import UserSecretsClient
    secrets = UserSecretsClient()
    FRED_API_KEY = secrets.get_secret("FRED_API_KEY")
    print("FRED API key loaded from Kaggle Secrets")
except:
    # Fallback for local testing (will fail in Kaggle if secret not set)
    FRED_API_KEY = os.environ.get('FRED_API_KEY')
    if FRED_API_KEY:
        print("FRED API key loaded from environment")
    else:
        raise KeyError("FRED_API_KEY not found in Kaggle Secrets or environment")

## 1. Data Fetching and Base Features

In [ ]:
def fetch_base_features():
    """
    Fetch base features from yfinance and FRED.
    Returns DataFrame with 19 base features + gold_return_next target.
    """
    print("Fetching base features...")
    
    fred = Fred(api_key=FRED_API_KEY)
    
    # Fetch FRED data
    start_date = '2014-06-01'
    
    dfii10 = fred.get_series('DFII10', observation_start=start_date)  # Real rate
    dgs10 = fred.get_series('DGS10', observation_start=start_date)    # 10Y yield
    dgs2 = fred.get_series('DGS2', observation_start=start_date)      # 2Y yield
    t10yie = fred.get_series('T10YIE', observation_start=start_date)  # Inflation expectation
    vix = fred.get_series('VIXCLS', observation_start=start_date)     # VIX
    
    # Create base DataFrame from FRED data
    df = pd.DataFrame({
        'real_rate_real_rate': dfii10,
        'yield_curve_dgs10': dgs10,
        'yield_curve_dgs2': dgs2,
        'inflation_expectation_inflation_expectation': t10yie,
        'vix_vix': vix
    })
    
    # Fetch Yahoo Finance data
    print("  Fetching Yahoo Finance data...")
    
    tickers = {
        'DX-Y.NYB': 'dxy',
        'GLD': 'technical_gld',
        'GC=F': 'gc',
        'SI=F': 'cross_asset_silver',
        'HG=F': 'cross_asset_copper',
        '^GSPC': 'cross_asset_sp500',
        'CNY=X': 'cny_demand_cny'
    }
    
    yf_data = {}
    for ticker, name in tickers.items():
        data = yf.download(ticker, start=start_date, progress=False)
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)
        yf_data[name] = data
    
    # Add DXY
    df['dxy_dxy'] = yf_data['dxy']['Close']
    
    # Add GLD OHLCV
    df['technical_gld_open'] = yf_data['technical_gld']['Open']
    df['technical_gld_high'] = yf_data['technical_gld']['High']
    df['technical_gld_low'] = yf_data['technical_gld']['Low']
    df['technical_gld_close'] = yf_data['technical_gld']['Close']
    df['technical_gld_volume'] = yf_data['technical_gld']['Volume']
    
    # Add cross-asset
    df['cross_asset_silver_close'] = yf_data['cross_asset_silver']['Close']
    df['cross_asset_copper_close'] = yf_data['cross_asset_copper']['Close']
    df['cross_asset_sp500_close'] = yf_data['cross_asset_sp500']['Close']
    
    # Add CNY
    df['cny_demand_cny_usd'] = yf_data['cny_demand_cny']['Close']
    
    # Add ETF flow features (using GLD)
    df['etf_flow_gld_volume'] = yf_data['technical_gld']['Volume']
    df['etf_flow_gld_close'] = yf_data['technical_gld']['Close']
    df['etf_flow_volume_ma20'] = df['etf_flow_gld_volume'].rolling(20).mean()
    
    # Add yield spread
    df['yield_curve_yield_spread'] = df['yield_curve_dgs10'] - df['yield_curve_dgs2']
    
    # Add target: next-day gold return
    gc_close = yf_data['gc']['Close']
    gold_return = gc_close.pct_change() * 100  # Percentage
    df['gold_return_next'] = gold_return.shift(-1)  # Next-day return
    
    # Forward-fill missing values (up to 3 days)
    df = df.ffill(limit=3)
    
    # Drop NaN
    df = df.dropna()
    
    print(f"  Base features loaded: {df.shape[0]} rows, {df.shape[1]} columns")
    print(f"  Date range: {df.index.min().date()} to {df.index.max().date()}")
    
    return df

base_df = fetch_base_features()
print(f"\nBase features shape: {base_df.shape}")
print(f"Columns: {list(base_df.columns)}")

## 2. Submodel Output Generation

Generate outputs from 7 HMM-based submodels using parameters from Phase 2.

In [ ]:
# === VIX Submodel ===
def generate_vix_features(base_df):
    """
    Generate VIX submodel features.
    Based on vix_1 parameters (3-component HMM on [vix_change, vix_vol_5d]).
    """
    print("Generating VIX submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['vix'] = base_df['vix_vix']
    
    # Compute derived features
    df['vix_change'] = df['vix'].diff()
    df['vix_vol_5d'] = df['vix_change'].rolling(5).std()
    df = df.dropna()
    
    # Train 3-component HMM
    X = df[['vix_change', 'vix_vol_5d']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=3, covariance_type='full', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 3), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify high-variance state
    state_vars = [model.covars_[i][0, 0] for i in range(3)]
    high_var_state = np.argmax(state_vars)
    
    # Compute features
    output = pd.DataFrame(index=base_df.index)
    output['vix_regime_probability'] = probs_df.iloc[:, high_var_state].reindex(base_df.index).fillna(0)
    
    # Mean reversion z-score
    vix_reindexed = df['vix'].reindex(base_df.index).fillna(method='ffill')
    vix_ma = vix_reindexed.rolling(20).mean()
    vix_std = vix_reindexed.rolling(60).std()
    output['vix_mean_reversion_z'] = ((vix_reindexed - vix_ma) / vix_std).clip(-4, 4).fillna(0)
    
    # Persistence
    vix_change_reindexed = df['vix_change'].reindex(base_df.index).fillna(0)
    output['vix_persistence'] = vix_change_reindexed.rolling(5).apply(lambda x: (x > 0).sum() / 5).fillna(0)
    
    print(f"  VIX features shape: {output.shape}")
    return output

vix_features = generate_vix_features(base_df)

In [ ]:
# === Technical Submodel ===
def generate_technical_features(base_df):
    """
    Generate technical submodel features.
    Based on technical_1 parameters (2-component HMM on [log_return, atr]).
    """
    print("Generating technical submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['close'] = base_df['technical_gld_close']
    df['high'] = base_df['technical_gld_high']
    df['low'] = base_df['technical_gld_low']
    
    # Compute features
    df['log_return'] = np.log(df['close'] / df['close'].shift(1))
    df['tr'] = np.maximum(df['high'] - df['low'], 
                          np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(14).mean()
    df = df.dropna()
    
    # Train 2-component HMM
    X = df[['log_return', 'atr']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=2, covariance_type='full', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 2), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify uptrend state (positive mean return)
    state_means = [model.means_[i][0] for i in range(2)]
    uptrend_state = np.argmax(state_means)
    
    output = pd.DataFrame(index=base_df.index)
    output['tech_trend_regime_prob'] = probs_df.iloc[:, uptrend_state].reindex(base_df.index).fillna(0)
    
    # Mean reversion z-score (RSI-based)
    close_reindexed = df['close'].reindex(base_df.index).fillna(method='ffill')
    delta = close_reindexed.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    rsi_ma = rsi.rolling(20).mean()
    rsi_std = rsi.rolling(60).std()
    output['tech_mean_reversion_z'] = ((rsi - rsi_ma) / rsi_std).clip(-4, 4).fillna(0)
    
    # Volatility regime (ATR percentile)
    atr_reindexed = df['atr'].reindex(base_df.index).fillna(method='ffill')
    atr_percentile = atr_reindexed.rolling(60).apply(lambda x: (x.iloc[-1] > x).sum() / len(x))
    output['tech_volatility_regime'] = atr_percentile.fillna(0.5)
    
    print(f"  Technical features shape: {output.shape}")
    return output

technical_features = generate_technical_features(base_df)

In [ ]:
# === Cross-Asset Submodel ===
def generate_cross_asset_features(base_df):
    """
    Generate cross-asset submodel features.
    Based on cross_asset_1 parameters.
    """
    print("Generating cross-asset submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['gold'] = base_df['technical_gld_close']
    df['silver'] = base_df['cross_asset_silver_close']
    df['sp500'] = base_df['cross_asset_sp500_close']
    
    # Compute features
    df['gs_ratio'] = df['gold'] / df['silver']
    df['gs_ratio_change'] = df['gs_ratio'].pct_change()
    df['sp500_return'] = df['sp500'].pct_change()
    df = df.dropna()
    
    # Train 2-component HMM
    X = df[['gs_ratio_change', 'sp500_return']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=2, covariance_type='diag', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 2), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify risk-off state (negative SP500 return mean)
    state_means_sp500 = [model.means_[i][1] for i in range(2)]
    risk_off_state = np.argmin(state_means_sp500)
    
    output = pd.DataFrame(index=base_df.index)
    output['xasset_regime_prob'] = probs_df.iloc[:, risk_off_state].reindex(base_df.index).fillna(0)
    
    # Recession signal (SP500 below 200-day MA)
    sp500_reindexed = df['sp500'].reindex(base_df.index).fillna(method='ffill')
    sp500_ma200 = sp500_reindexed.rolling(200).mean()
    output['xasset_recession_signal'] = (sp500_reindexed < sp500_ma200).astype(int).fillna(0)
    
    # Divergence (G/S ratio z-score)
    gs_ratio_reindexed = df['gs_ratio'].reindex(base_df.index).fillna(method='ffill')
    gs_ma = gs_ratio_reindexed.rolling(60).mean()
    gs_std = gs_ratio_reindexed.rolling(60).std()
    output['xasset_divergence'] = ((gs_ratio_reindexed - gs_ma) / gs_std).clip(-4, 4).fillna(0)
    
    print(f"  Cross-asset features shape: {output.shape}")
    return output

cross_asset_features = generate_cross_asset_features(base_df)

In [ ]:
# === Yield Curve Submodel ===
def generate_yield_curve_features(base_df):
    """
    Generate yield curve submodel features.
    Note: yc_regime_prob is excluded (constant).
    """
    print("Generating yield curve submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['spread'] = base_df['yield_curve_yield_spread']
    
    # Compute features
    df['spread_velocity'] = df['spread'].diff()
    df['spread_ma'] = df['spread'].rolling(20).mean()
    df['spread_std'] = df['spread'].rolling(60).std()
    df = df.dropna()
    
    output = pd.DataFrame(index=base_df.index)
    
    # Spread velocity z-score
    velocity_reindexed = df['spread_velocity'].reindex(base_df.index).fillna(0)
    velocity_ma = velocity_reindexed.rolling(60).mean()
    velocity_std = velocity_reindexed.rolling(60).std()
    output['yc_spread_velocity_z'] = ((velocity_reindexed - velocity_ma) / velocity_std).clip(-4, 4).fillna(0)
    
    # Curvature z-score (2nd derivative)
    spread_reindexed = df['spread'].reindex(base_df.index).fillna(method='ffill')
    curvature = spread_reindexed.diff().diff()
    curvature_ma = curvature.rolling(60).mean()
    curvature_std = curvature.rolling(60).std()
    output['yc_curvature_z'] = ((curvature - curvature_ma) / curvature_std).clip(-4, 4).fillna(0)
    
    print(f"  Yield curve features shape: {output.shape}")
    return output

yield_curve_features = generate_yield_curve_features(base_df)

In [ ]:
# === ETF Flow Submodel ===
def generate_etf_flow_features(base_df):
    """
    Generate ETF flow submodel features.
    """
    print("Generating ETF flow submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['volume'] = base_df['etf_flow_gld_volume']
    df['close'] = base_df['etf_flow_gld_close']
    
    # Compute features
    df['volume_change'] = df['volume'].pct_change()
    df['price_return'] = df['close'].pct_change()
    df = df.dropna()
    
    # Train 2-component HMM
    X = df[['volume_change', 'price_return']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=2, covariance_type='diag', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 2), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify high-flow state
    state_means_volume = [model.means_[i][0] for i in range(2)]
    high_flow_state = np.argmax(state_means_volume)
    
    output = pd.DataFrame(index=base_df.index)
    output['etf_regime_prob'] = probs_df.iloc[:, high_flow_state].reindex(base_df.index).fillna(0)
    
    # Capital intensity (volume * price)
    volume_reindexed = df['volume'].reindex(base_df.index).fillna(method='ffill')
    close_reindexed = df['close'].reindex(base_df.index).fillna(method='ffill')
    capital = volume_reindexed * close_reindexed
    capital_ma = capital.rolling(20).mean()
    capital_std = capital.rolling(60).std()
    output['etf_capital_intensity'] = ((capital - capital_ma) / capital_std).clip(-4, 4).fillna(0)
    
    # Price-volume divergence
    price_z = ((close_reindexed - close_reindexed.rolling(20).mean()) / close_reindexed.rolling(60).std()).fillna(0)
    volume_z = ((volume_reindexed - volume_reindexed.rolling(20).mean()) / volume_reindexed.rolling(60).std()).fillna(0)
    output['etf_pv_divergence'] = (price_z - volume_z).clip(-4, 4).fillna(0)
    
    print(f"  ETF flow features shape: {output.shape}")
    return output

etf_flow_features = generate_etf_flow_features(base_df)

In [ ]:
# === Inflation Expectation Submodel ===
def generate_inflation_expectation_features(base_df):
    """
    Generate inflation expectation submodel features.
    """
    print("Generating inflation expectation submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['ie'] = base_df['inflation_expectation_inflation_expectation']
    
    # Compute features
    df['ie_change'] = df['ie'].diff()
    df['ie_vol_5d'] = df['ie_change'].rolling(5).std()
    df = df.dropna()
    
    # Train 2-component HMM
    X = df[['ie_change', 'ie_vol_5d']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=2, covariance_type='full', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 2), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify high-variance state
    state_vars = [model.covars_[i][0, 0] for i in range(2)]
    high_var_state = np.argmax(state_vars)
    
    output = pd.DataFrame(index=base_df.index)
    output['ie_regime_prob'] = probs_df.iloc[:, high_var_state].reindex(base_df.index).fillna(0)
    
    # Anchoring z-score
    ie_vol_reindexed = df['ie_vol_5d'].reindex(base_df.index).fillna(method='ffill')
    ie_vol_ma = ie_vol_reindexed.rolling(60).mean()
    ie_vol_std = ie_vol_reindexed.rolling(60).std()
    output['ie_anchoring_z'] = ((ie_vol_reindexed - ie_vol_ma) / ie_vol_std).clip(-4, 4).fillna(0)
    
    # IE-gold sensitivity (correlation)
    ie_change_reindexed = df['ie_change'].reindex(base_df.index).fillna(0)
    gold_return = base_df['technical_gld_close'].pct_change()
    rolling_corr = ie_change_reindexed.rolling(5).corr(gold_return)
    corr_ma = rolling_corr.rolling(60).mean()
    corr_std = rolling_corr.rolling(60).std()
    output['ie_gold_sensitivity_z'] = ((rolling_corr - corr_ma) / corr_std).clip(-4, 4).fillna(0)
    
    print(f"  Inflation expectation features shape: {output.shape}")
    return output

inflation_expectation_features = generate_inflation_expectation_features(base_df)

In [ ]:
# === CNY Demand Submodel ===
def generate_cny_demand_features(base_df):
    """
    Generate CNY demand submodel features.
    """
    print("Generating CNY demand submodel features...")
    
    df = pd.DataFrame(index=base_df.index)
    df['cny_usd'] = base_df['cny_demand_cny_usd']
    
    # Compute features
    df['cny_change'] = df['cny_usd'].pct_change()
    df['cny_vol'] = df['cny_change'].rolling(10).std()
    df = df.dropna()
    
    # Train 2-component HMM
    X = df[['cny_change', 'cny_vol']].values
    valid_mask = np.isfinite(X).all(axis=1)
    X_clean = X[valid_mask]
    
    model = hmmlearn_hmm.GaussianHMM(n_components=2, covariance_type='diag', n_iter=100, random_state=42)
    model.fit(X_clean)
    
    # Predict probabilities
    probs = np.full((len(df), 2), np.nan)
    probs[valid_mask] = model.predict_proba(X[valid_mask])
    probs_df = pd.DataFrame(probs, index=df.index).fillna(method='ffill').fillna(method='bfill')
    
    # Identify high-volatility state
    state_vars = [model.covars_[i][1] for i in range(2)]
    high_vol_state = np.argmax(state_vars)
    
    output = pd.DataFrame(index=base_df.index)
    output['cny_regime_prob'] = probs_df.iloc[:, high_vol_state].reindex(base_df.index).fillna(0)
    
    # Momentum z-score
    cny_change_reindexed = df['cny_change'].reindex(base_df.index).fillna(0)
    momentum = cny_change_reindexed.rolling(20).mean()
    momentum_ma = momentum.rolling(60).mean()
    momentum_std = momentum.rolling(60).std()
    output['cny_momentum_z'] = ((momentum - momentum_ma) / momentum_std).clip(-4, 4).fillna(0)
    
    # Volatility regime z-score
    cny_vol_reindexed = df['cny_vol'].reindex(base_df.index).fillna(method='ffill')
    vol_ma = cny_vol_reindexed.rolling(60).mean()
    vol_std = cny_vol_reindexed.rolling(60).std()
    output['cny_vol_regime_z'] = ((cny_vol_reindexed - vol_ma) / vol_std).clip(-4, 4).fillna(0)
    
    print(f"  CNY demand features shape: {output.shape}")
    return output

cny_demand_features = generate_cny_demand_features(base_df)

## 3. Merge All Features

In [ ]:
# Separate target from base features
target = base_df['gold_return_next'].copy()
base_features = base_df.drop(columns=['gold_return_next'])

# Merge all features
all_features = base_features.copy()
all_features = all_features.join(vix_features, how='inner')
all_features = all_features.join(technical_features, how='inner')
all_features = all_features.join(cross_asset_features, how='inner')
all_features = all_features.join(yield_curve_features, how='inner')
all_features = all_features.join(etf_flow_features, how='inner')
all_features = all_features.join(inflation_expectation_features, how='inner')
all_features = all_features.join(cny_demand_features, how='inner')

# Align target
target = target.reindex(all_features.index)

# Drop NaN rows
valid_mask = ~(all_features.isna().any(axis=1) | target.isna())
all_features = all_features[valid_mask]
target = target[valid_mask]

print(f"\nMerged data:")
print(f"  Features shape: {all_features.shape}")
print(f"  Target shape: {target.shape}")
print(f"  Date range: {all_features.index.min().date()} to {all_features.index.max().date()}")
print(f"\nFeature columns ({len(all_features.columns)}):")
for i, col in enumerate(all_features.columns, 1):
    print(f"  {i:2d}. {col}")

## 4. Data Split

In [ ]:
# Split 70/15/15 (time-series order)
n = len(all_features)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

X_train = all_features.iloc[:train_end].values
y_train = target.iloc[:train_end].values
dates_train = all_features.iloc[:train_end].index

X_val = all_features.iloc[train_end:val_end].values
y_val = target.iloc[train_end:val_end].values
dates_val = all_features.iloc[train_end:val_end].index

X_test = all_features.iloc[val_end:].values
y_test = target.iloc[val_end:].values
dates_test = all_features.iloc[val_end:].index

print(f"\nData splits:")
print(f"  Train: {len(X_train)} samples ({len(X_train)/n*100:.1f}%)")
print(f"  Val:   {len(X_val)} samples ({len(X_val)/n*100:.1f}%)")
print(f"  Test:  {len(X_test)} samples ({len(X_test)/n*100:.1f}%)")

# Compute test set up/down ratio
test_up = (y_test > 0).sum()
test_down = (y_test < 0).sum()
test_zero = (y_test == 0).sum()
print(f"\nTest set composition:")
print(f"  Up days: {test_up} ({test_up/len(y_test)*100:.1f}%)")
print(f"  Down days: {test_down} ({test_down/len(y_test)*100:.1f}%)")
print(f"  Zero days: {test_zero}")
print(f"  Naive always-up DA: {test_up / (test_up + test_down) * 100:.1f}%")

## 5. Helper Functions

In [ ]:
def compute_metrics(y_pred, y_true, conf_threshold=0.005):
    """
    Compute all evaluation metrics.
    
    Args:
        y_pred: Predicted returns (%)
        y_true: Actual returns (%)
        conf_threshold: Confidence threshold for high-confidence DA
    
    Returns:
        dict of metrics
    """
    # MAE
    mae = np.mean(np.abs(y_pred - y_true))
    
    # Direction Accuracy (exclude zeros)
    nonzero_mask = (y_true != 0) & (y_pred != 0)
    if nonzero_mask.sum() > 0:
        da = np.mean(np.sign(y_pred[nonzero_mask]) == np.sign(y_true[nonzero_mask]))
    else:
        da = 0.5
    
    # High-Confidence DA
    hc_mask = (np.abs(y_pred) > conf_threshold) & nonzero_mask
    if hc_mask.sum() >= 0.2 * len(y_pred):
        hc_da = np.mean(np.sign(y_pred[hc_mask]) == np.sign(y_true[hc_mask]))
        hc_coverage = hc_mask.sum() / len(y_pred)
    else:
        hc_da = 0.0
        hc_coverage = hc_mask.sum() / len(y_pred)
    
    # Sharpe Ratio (with 5bps transaction cost per day)
    cost_pct = 5.0 / 100.0  # 5 bps = 0.05%
    strategy_returns = np.sign(y_pred) * y_true
    net_returns = strategy_returns - cost_pct
    
    if len(net_returns) > 1 and np.std(net_returns) > 0:
        sharpe = (np.mean(net_returns) / np.std(net_returns)) * np.sqrt(252)
    else:
        sharpe = 0.0
    
    return {
        'mae': float(mae),
        'direction_accuracy': float(da),
        'high_confidence_da': float(hc_da),
        'sharpe_ratio': float(sharpe),
        'hc_coverage': float(hc_coverage),
        'n_samples': len(y_pred)
    }

def composite_eval(y_pred, dtrain):
    """
    Custom evaluation metric for XGBoost early stopping.
    Lower is better.
    """
    y_true = dtrain.get_label()
    
    # MAE component
    mae = np.mean(np.abs(y_pred - y_true))
    
    # DA component
    nonzero = (y_true != 0) & (y_pred != 0)
    if nonzero.sum() > 0:
        da = np.mean(np.sign(y_pred[nonzero]) == np.sign(y_true[nonzero]))
    else:
        da = 0.5
    
    # Composite: lower is better
    score = mae - 0.5 * da
    
    return 'composite', float(score)

print("Helper functions defined.")

## 6. Optuna Hyperparameter Optimization

In [ ]:
def optuna_objective(trial):
    """
    Optuna objective function.
    Maximizes weighted composite of DA, HC-DA, MAE, and Sharpe.
    """
    # Sample hyperparameters
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 6),
        'min_child_weight': trial.suggest_int('min_child_weight', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.8),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 5.0, log=True),
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.05, log=True),
        'gamma': trial.suggest_float('gamma', 0.0, 2.0),
        'tree_method': 'hist',
        'disable_default_eval_metric': 1,
        'verbosity': 0,
        'seed': 42 + trial.number,
    }
    
    penalty_factor = trial.suggest_float('directional_penalty', 1.5, 5.0)
    conf_threshold = trial.suggest_float('confidence_threshold', 0.002, 0.015)
    
    # Custom objective with directional penalty
    def obj_fn(y_pred, dtrain):
        y_true = dtrain.get_label()
        sign_agree = (y_pred * y_true) > 0
        penalty = np.where(sign_agree, 1.0, penalty_factor)
        residual = y_pred - y_true
        grad = penalty * np.sign(residual)
        hess = penalty * np.ones_like(y_pred)
        return grad, hess
    
    # Train
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    
    try:
        bst = xgb.train(
            params, dtrain,
            num_boost_round=1000,
            obj=obj_fn,
            evals=[(dval, 'val')],
            custom_metric=composite_eval,
            early_stopping_rounds=50,
            verbose_eval=False,
        )
    except Exception as e:
        print(f"Trial {trial.number} failed: {e}")
        return -np.inf
    
    # Predict on validation set
    val_pred = bst.predict(dval)
    
    # Check for degenerate predictions
    if np.std(val_pred) < 0.001:
        print(f"Trial {trial.number}: degenerate predictions (std={np.std(val_pred):.6f})")
        return -np.inf
    
    # Compute metrics
    metrics = compute_metrics(val_pred, y_val, conf_threshold)
    
    # Normalize metrics to [0, 1]
    sharpe_norm = np.clip((metrics['sharpe_ratio'] + 3.0) / 6.0, 0.0, 1.0)
    da_norm = np.clip((metrics['direction_accuracy'] - 0.3) / 0.4, 0.0, 1.0)
    mae_norm = np.clip((1.0 - metrics['mae']) / 0.5, 0.0, 1.0)
    hc_da_norm = np.clip((metrics['high_confidence_da'] - 0.3) / 0.4, 0.0, 1.0) if metrics['high_confidence_da'] > 0 else 0.0
    
    # Weighted composite (Sharpe 40%, DA 25%, HC-DA 20%, MAE 15%)
    objective_value = (
        0.40 * sharpe_norm +
        0.25 * da_norm +
        0.20 * hc_da_norm +
        0.15 * mae_norm
    )
    
    # Log metrics
    trial.set_user_attr('val_mae', metrics['mae'])
    trial.set_user_attr('val_da', metrics['direction_accuracy'])
    trial.set_user_attr('val_hc_da', metrics['high_confidence_da'])
    trial.set_user_attr('val_sharpe', metrics['sharpe_ratio'])
    trial.set_user_attr('val_hc_coverage', metrics['hc_coverage'])
    trial.set_user_attr('n_estimators', int(bst.best_iteration + 1))
    
    return objective_value

print("\n=== Running Optuna HPO (50 trials, 4-hour timeout) ===")

study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42)
)

study.optimize(
    optuna_objective,
    n_trials=50,
    timeout=14400,  # 4 hours
    show_progress_bar=True
)

print(f"\nOptuna optimization complete!")
print(f"  Best objective value: {study.best_value:.6f}")
print(f"  Best trial: #{study.best_trial.number}")
print(f"  Trials completed: {len(study.trials)}")
print(f"\nBest hyperparameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

print(f"\nBest trial metrics:")
for key, value in study.best_trial.user_attrs.items():
    print(f"  {key}: {value}")

## 7. Final Model Training

In [ ]:
print("\n=== Training final model with best hyperparameters ===")

best_params = study.best_params.copy()
penalty_factor = best_params.pop('directional_penalty')
conf_threshold = best_params.pop('confidence_threshold')

# Add fixed parameters
best_params.update({
    'tree_method': 'hist',
    'disable_default_eval_metric': 1,
    'verbosity': 0,
    'seed': 42,
})

# Custom objective
def final_obj_fn(y_pred, dtrain):
    y_true = dtrain.get_label()
    sign_agree = (y_pred * y_true) > 0
    penalty = np.where(sign_agree, 1.0, penalty_factor)
    residual = y_pred - y_true
    grad = penalty * np.sign(residual)
    hess = penalty * np.ones_like(y_pred)
    return grad, hess

# Train
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

final_model = xgb.train(
    best_params, dtrain,
    num_boost_round=1000,
    obj=final_obj_fn,
    evals=[(dval, 'val')],
    custom_metric=composite_eval,
    early_stopping_rounds=50,
    verbose_eval=10,
)

print(f"\nFinal model trained!")
print(f"  Best iteration: {final_model.best_iteration}")
print(f"  Total boosting rounds: {final_model.best_iteration + 1}")

## 8. Evaluation on All Splits

In [ ]:
print("\n=== Evaluating on all splits ===")

# Predict
train_pred = final_model.predict(dtrain)
val_pred = final_model.predict(dval)
test_pred = final_model.predict(dtest)

# Compute metrics
train_metrics = compute_metrics(train_pred, y_train, conf_threshold)
val_metrics = compute_metrics(val_pred, y_val, conf_threshold)
test_metrics = compute_metrics(test_pred, y_test, conf_threshold)

print("\nTrain metrics:")
for key, value in train_metrics.items():
    print(f"  {key}: {value:.4f}")

print("\nValidation metrics:")
for key, value in val_metrics.items():
    print(f"  {key}: {value:.4f}")

print("\nTest metrics:")
for key, value in test_metrics.items():
    print(f"  {key}: {value:.4f}")

# Overfit ratios
mae_overfit_val = val_metrics['mae'] / (train_metrics['mae'] + 1e-10)
mae_overfit_test = test_metrics['mae'] / (train_metrics['mae'] + 1e-10)
da_gap_test = (train_metrics['direction_accuracy'] - test_metrics['direction_accuracy']) * 100

print("\nOverfit diagnostics:")
print(f"  MAE val/train ratio: {mae_overfit_val:.3f}")
print(f"  MAE test/train ratio: {mae_overfit_test:.3f}")
print(f"  DA train-test gap: {da_gap_test:.2f}pp")

# Prediction distribution
print("\nTest prediction distribution:")
print(f"  Mean: {np.mean(test_pred):.6f}")
print(f"  Std: {np.std(test_pred):.6f}")
print(f"  Min: {np.min(test_pred):.6f}")
print(f"  Max: {np.max(test_pred):.6f}")
print(f"  % positive predictions: {(test_pred > 0).sum() / len(test_pred) * 100:.1f}%")

# Naive always-up comparison
naive_da = test_up / (test_up + test_down)
print(f"\nNaive always-up DA: {naive_da:.4f}")
print(f"Model DA improvement: {(test_metrics['direction_accuracy'] - naive_da) * 100:.2f}pp")

## 9. Feature Importance

In [ ]:
# Get feature importance
importance_dict = final_model.get_score(importance_type='gain')

# Map feature indices to names
feature_names = list(all_features.columns)
importance_list = [(feature_names[int(k.replace('f', ''))], v) 
                   for k, v in importance_dict.items()]
importance_list.sort(key=lambda x: x[1], reverse=True)

print("\nTop 20 feature importances:")
for i, (name, score) in enumerate(importance_list[:20], 1):
    print(f"  {i:2d}. {name:40s}: {score:.0f}")

# Check submodel feature usage
submodel_prefixes = ['vix_', 'tech_', 'xasset_', 'yc_', 'etf_', 'ie_', 'cny_']
submodel_importance = {prefix: 0 for prefix in submodel_prefixes}

for name, score in importance_list:
    for prefix in submodel_prefixes:
        if name.startswith(prefix):
            submodel_importance[prefix] += score
            break

print("\nSubmodel feature importance (total gain):")
for prefix, score in sorted(submodel_importance.items(), key=lambda x: x[1], reverse=True):
    print(f"  {prefix:10s}: {score:.0f}")

## 10. Save Results

In [ ]:
print("\n=== Saving results ===")

# 1. training_result.json
result = {
    'feature': 'meta_model',
    'attempt': 1,
    'timestamp': datetime.now().isoformat(),
    'model_type': 'XGBoost',
    'n_features': all_features.shape[1],
    'best_params': {
        **best_params,
        'directional_penalty': penalty_factor,
        'confidence_threshold': conf_threshold,
        'n_estimators_used': int(final_model.best_iteration + 1)
    },
    'metrics': {
        'train': train_metrics,
        'val': val_metrics,
        'test': test_metrics
    },
    'overfit_ratios': {
        'mae_val_train': float(mae_overfit_val),
        'mae_test_train': float(mae_overfit_test),
        'da_train_test_gap_pp': float(da_gap_test)
    },
    'feature_importance_top20': dict(importance_list[:20]),
    'submodel_importance': submodel_importance,
    'naive_always_up_da_test': float(naive_da),
    'optuna_summary': {
        'n_trials': len(study.trials),
        'best_trial': study.best_trial.number,
        'best_value': float(study.best_value),
        'trial_details': [
            {
                'trial': t.number,
                'value': float(t.value) if t.value is not None else None,
                'params': t.params,
                'user_attrs': t.user_attrs
            }
            for t in study.trials
        ]
    },
    'prediction_distribution': {
        'test_pct_positive': float((test_pred > 0).sum() / len(test_pred)),
        'test_pred_std': float(np.std(test_pred)),
        'test_pred_mean': float(np.mean(test_pred))
    }
}

with open('training_result.json', 'w') as f:
    json.dump(result, f, indent=2)
print("  Saved: training_result.json")

# 2. model.json
final_model.save_model('model.json')
print("  Saved: model.json")

# 3. predictions_test.csv
predictions_df = pd.DataFrame({
    'date': list(dates_train) + list(dates_val) + list(dates_test),
    'split': ['train'] * len(dates_train) + ['val'] * len(dates_val) + ['test'] * len(dates_test),
    'prediction': np.concatenate([train_pred, val_pred, test_pred]),
    'actual': np.concatenate([y_train, y_val, y_test])
})
predictions_df['direction_correct'] = np.sign(predictions_df['prediction']) == np.sign(predictions_df['actual'])
predictions_df['high_confidence'] = np.abs(predictions_df['prediction']) > conf_threshold
predictions_df.to_csv('predictions_test.csv', index=False)
print("  Saved: predictions_test.csv")

# 4. submodel_output.csv (for compatibility)
full_pred = final_model.predict(xgb.DMatrix(all_features.values))
submodel_output = pd.DataFrame({
    'meta_prediction': full_pred
}, index=all_features.index)
submodel_output.to_csv('submodel_output.csv')
print("  Saved: submodel_output.csv")

print("\n" + "=" * 80)
print("Meta-Model Training Complete!")
print("=" * 80)
print(f"\nTest Results:")
print(f"  Direction Accuracy: {test_metrics['direction_accuracy']*100:.2f}% (target: >56%)")
print(f"  High-Confidence DA: {test_metrics['high_confidence_da']*100:.2f}% (target: >60%)")
print(f"  MAE: {test_metrics['mae']:.4f}% (target: <0.75%)")
print(f"  Sharpe Ratio: {test_metrics['sharpe_ratio']:.3f} (target: >0.8)")

# Check if targets met
targets_met = [
    test_metrics['direction_accuracy'] > 0.56,
    test_metrics['high_confidence_da'] > 0.60,
    test_metrics['mae'] < 0.75,
    test_metrics['sharpe_ratio'] > 0.8
]

print(f"\nTargets met: {sum(targets_met)}/4")
if all(targets_met):
    print("\n🎉 ALL TARGETS MET! Phase 3 complete.")
else:
    print("\n⚠️  Some targets not met. Attempt 2 may be needed.")